In [1]:
import os

# Set CUDA_VISIBLE_DEVICES to use only the first GPU (index 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
os.environ["OMP_NUM_THREADS"] = "16"
os.environ["MKL_NUM_THREADS"] = "16"

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split


# need to make sure that we're reading from the current folder -> replacing pre-installed tabpfn
import sys
sys.path.insert(0, "/home/sergazinov/chunk-tabpfn/src")
from tabpfn.classifier import TabPFNClassifier

from tabpfn.extensions.replace_chunk_attn_computation import enable_chunked_attention

In [2]:
# default params for exp-s, min recommended chunk_size = 256 -> otherwise, too slow
enable_chunked_attention(chunk_size=512, batch_size=1024)

[TabPFN] ▶ Enabled chunked self‑attention (chunk_size=512, batch_size=1024)


In [3]:
cov = fetch_openml(data_id=293, as_frame='auto',)
X, y = cov.data, cov.target

y = y.astype('int')
y[y==-1] = 0

X = pd.DataFrame(X.toarray())
print(X.shape, y.shape)

(581012, 54) (581012,)


In [4]:
# Example usage w/100K data, 50 features
X_train = X[:100_000].copy()
y_train = y[:100_000].copy()

X_test = X[200_000:210_000].copy()
y_test = y[200_000:210_000].copy()

In [5]:
# 100K , 50 features, 2min on A100
clf = TabPFNClassifier(device='cuda', ignore_pretraining_limits=True)
clf.fit(X_train, y_train)
prediction_probabilities = clf.predict_proba(X_test)
print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

/home/sergazinov/chunk-tabpfn/src/tabpfn/classifier.py:422: UserWarning: Number of samples 100000 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(
/home/sergazinov/.local/share/uv/python/cpython-3.11.11-linux-x86_64-gnu/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/home/sergazinov/.local/share/uv/python/cpython-3.11.11-linux-x86_64-gnu/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


ROC AUC: 0.8978702392317726
